In [1]:
import pickle
import os
import warnings
from tqdm import tqdm
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')

로그 스케일로 데이터 변환
- 지표간 차이
- 이전 STEP에서의 변화량

일반 스케일 데이터
- VOLUME/RSI 
- MA대신 MACD 지표 사용

가격데이터는 MID_PRICE만 사용

In [2]:
with open('C:\\Users\\user\\Documents\\GitHub\\crypto-scalping-RL-Agent\\data\\BTCUSD_1min.pkl', 'rb') as f:
    df_og = pickle.load(f)

In [3]:
df = df_og.loc['2022-11-14 05:15:00':]
df["mid_price"] = df['low']/2 + df['high']/2

df_og = df_og.loc['2022-11-14 05:15:00':]

In [4]:
df['log_returns'] = np.log(df['mid_price'] / df['mid_price'].shift(1))
df

,low,high,open,close,volume,mid_price,log_returns
time,,,,,,,
2022-11-14 05:15:00,15940.70,15951.72,15945.10,15941.61,37.486634,15946.210,NaN
2022-11-14 05:16:00,15936.86,15950.51,15943.82,15942.54,27.412980,15943.685,-0.000158
2022-11-14 05:17:00,15940.50,15957.37,15942.54,15950.75,30.118080,15948.935,0.000329
2022-11-14 05:18:00,15950.77,15969.00,15950.77,15968.94,16.315539,15959.885,0.000686
2022-11-14 05:19:00,15961.21,15973.60,15966.61,15965.72,43.274225,15967.405,0.000471
...,...,...,...,...,...,...,...
2023-11-14 05:11:00,36527.23,36538.29,36535.58,36535.43,0.551182,36532.760,0.000028
2023-11-14 05:12:00,36535.41,36538.32,36535.42,36536.52,1.442079,36536.865,0.000112
2023-11-14 05:13:00,36535.41,36549.62,36536.52,36535.42,5.654814,36542.515,0.000155


In [5]:
df['7min_MA'] = df['mid_price'].rolling(window=7).mean()
df['25min_MA'] = df['mid_price'].rolling(window=25).mean()
df['99min_MA'] = df['mid_price'].rolling(window=90).mean()
df['300min_MA'] = df['mid_price'].rolling(window=300).mean()
df['900min_MA'] = df['mid_price'].rolling(window=900).mean()


df['7_ema'] = df['mid_price'].ewm(span=7, adjust=False).mean()
df['25_ema'] = df['mid_price'].ewm(span=25, adjust=False).mean()
df['99_ema'] = df['mid_price'].ewm(span=90,adjust=False).mean()
df['300_ema'] = df['mid_price'].ewm(span=300,adjust=False).mean()
df['900_ema'] = df['mid_price'].ewm(span=900,adjust=False).mean()


df['log_7min_MA'] = df['log_returns'].rolling(window=7).mean()
df['log_25min_MA'] = df['log_returns'].rolling(window=25).mean()
df['log_99min_MA'] = df['log_returns'].rolling(window=90).mean()
df['log_300min_MA'] = df['log_returns'].rolling(window=300).mean()
df['log_900min_MA'] = df['log_returns'].rolling(window=900).mean()


df['log_7_ema'] = df['log_returns'].ewm(span=7, adjust=False).mean()
df['log_25_ema'] = df['log_returns'].ewm(span=25, adjust=False).mean()
df['log_99_ema'] = df['log_returns'].ewm(span=90,adjust=False).mean()
df['log_300_ema'] = df['log_returns'].ewm(span=300,adjust=False).mean()
df['log_900_ema'] = df['log_returns'].ewm(span=900,adjust=False).mean()

In [6]:
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    # 단기 및 장기 지수 이동 평균 계산
    df['Short_EMA'] = df['mid_price'].ewm(span=short_window, adjust=False).mean()
    df['Long_EMA'] = df['mid_price'].ewm(span=long_window, adjust=False).mean()

    # MACD 계산
    df['MACD'] = df['Short_EMA'] - df['Long_EMA']

    # 신호선 계산
    df['Signal_Line'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()

    return df[['MACD', 'Signal_Line']]

macd_df = calculate_macd(df)

macd_df

,MACD,Signal_Line
time,,
2022-11-14 05:15:00,0.000000,0.000000
2022-11-14 05:16:00,-0.201425,-0.040285
2022-11-14 05:17:00,0.061863,-0.019855
2022-11-14 05:18:00,1.140942,0.212304
2022-11-14 05:19:00,2.573258,0.684495
...,...,...
2023-11-14 05:11:00,11.503282,12.163212
2023-11-14 05:12:00,11.388726,12.008315
2023-11-14 05:13:00,11.619901,11.930632


In [7]:
df['obv'] = [0 for x in range(df.shape[0])]

for i in tqdm(range(1, len(df))):
    if df['close'][i] > df['close'][i - 1]:
        df['obv'][i] = df['obv'][i - 1] + df['volume'][i]
    elif df['close'][i] < df['close'][i - 1]:
        df['obv'][i] = df['obv'][i - 1] - df['volume'][i]
    else:
        df['obv'][i] = df['obv'][i - 1]

100%|██████████| 525254/525254 [00:37<00:00, 13985.12it/s]


In [8]:
df['obv'] /= 5000

In [9]:
df = df.dropna(subset=['mid_price'])

# Calculate price changes
delta = df['mid_price'].diff(1)

# Define the time period for RSI (e.g., 14)
rsi_period = 14

# Calculate gains and losses
gains = delta.where(delta > 0, 0)
losses = -delta.where(delta < 0, 0)

# Calculate average gains and losses
avg_gain = gains.rolling(window=rsi_period, min_periods=1).mean()
avg_loss = losses.rolling(window=rsi_period, min_periods=1).mean()

# Calculate relative strength (RS)
rs = avg_gain / avg_loss

# Calculate RSI
df['rsi'] = (100 - (100 / (1 + rs)))/100

In [10]:
df['vma'] = df['volume'].rolling(window=20).mean()

In [11]:
df = df.dropna()
df.head()

,low,high,open,close,volume,mid_price,log_returns,7min_MA,25min_MA,99min_MA,...,log_99_ema,log_300_ema,log_900_ema,Short_EMA,Long_EMA,MACD,Signal_Line,obv,rsi,vma
time,,,,,,,,,,,,,,,,,,,,,
2022-11-14 20:15:00,16216.84,16250.28,16223.65,16227.41,56.890409,16233.560,0.001802,16208.770000,16252.7574,16450.468444,...,-0.000228,-0.000111,-0.000055,16223.914163,16265.232826,-41.318663,-47.808444,-0.039246,0.444260,63.030146
2022-11-14 20:16:00,16219.96,16254.47,16226.80,16240.69,38.518180,16237.215,0.000225,16214.318571,16244.9290,16446.476500,...,-0.000218,-0.000109,-0.000054,16225.960445,16263.157431,-37.196986,-45.686152,-0.031542,0.447616,60.001493
2022-11-14 20:17:00,16237.95,16270.42,16237.95,16251.46,40.355329,16254.185,0.001045,16218.565714,16238.3728,16442.670222,...,-0.000190,-0.000101,-0.000052,16230.302685,16262.492807,-32.190122,-42.986946,-0.023471,0.427829,59.764465
2022-11-14 20:18:00,16245.76,16272.18,16248.66,16261.14,70.668109,16258.970,0.000294,16225.402857,16234.2640,16438.698111,...,-0.000180,-0.000098,-0.000051,16234.713041,16262.231858,-27.518817,-39.893321,-0.009338,0.437425,60.431603
2022-11-14 20:19:00,16254.12,16286.69,16260.00,16274.72,59.737850,16270.405,0.000703,16236.347857,16232.0018,16435.054556,...,-0.000160,-0.000093,-0.000050,16240.204111,16262.837276,-22.633165,-36.441289,0.002610,0.524746,60.630043


In [12]:
df.columns

Index(['low', 'high', 'open', 'close', 'volume', 'mid_price', 'log_returns',
       '7min_MA', '25min_MA', '99min_MA', '300min_MA', '900min_MA', '7_ema',
       '25_ema', '99_ema', '300_ema', '900_ema', 'log_7min_MA', 'log_25min_MA',
       'log_99min_MA', 'log_300min_MA', 'log_900min_MA', 'log_7_ema',
       'log_25_ema', 'log_99_ema', 'log_300_ema', 'log_900_ema', 'Short_EMA',
       'Long_EMA', 'MACD', 'Signal_Line', 'obv', 'rsi', 'vma'],
      dtype='object')

In [13]:
def pct_change_with_zero_handling(df):
    # Step 1: Calculate regular percentage change
    pct_change_result = df.pct_change()
    inf_mask = np.isinf(pct_change_result)
    pct_change_result[inf_mask] = 0
    return pct_change_result

price_col =['volume', 'mid_price', 'log_returns',
       '7min_MA', '25min_MA', '99min_MA', '300min_MA', '900min_MA', '7_ema',
       '25_ema', '99_ema', '300_ema', '900_ema', 'Short_EMA',
       'Long_EMA', 'vma']

df[price_col] = pct_change_with_zero_handling(df[price_col])

np.isinf(df[price_col].values).sum()

0

In [14]:
df["vol_pct"] = df[['volume']].pct_change()

In [15]:
df['MACD-SIGNAL'] = df['MACD'] - df['Signal_Line']

In [16]:
df_final = df.drop(['Short_EMA', "Long_EMA", 'high', 'low', 'open', 'close'], axis=1)
df = df.dropna()
df_final.head()

,volume,mid_price,log_returns,7min_MA,25min_MA,99min_MA,300min_MA,900min_MA,7_ema,25_ema,...,log_99_ema,log_300_ema,log_900_ema,MACD,Signal_Line,obv,rsi,vma,vol_pct,MACD-SIGNAL
time,,,,,,,,,,,,,,,,,,,,,
2022-11-14 20:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.000228,-0.000111,-0.000055,-41.318663,-47.808444,-0.039246,0.444260,NaN,NaN,6.489781
2022-11-14 20:16:00,-0.322941,0.000225,-0.875063,0.000342,-0.000482,-0.000243,-0.000081,0.00002,0.000336,-0.000118,...,-0.000218,-0.000109,-0.000054,-37.196986,-45.686152,-0.031542,0.447616,-0.048051,NaN,8.489166
2022-11-14 20:17:00,0.047696,0.001045,3.640008,0.000262,-0.000404,-0.000231,-0.000075,0.00002,0.000513,-0.000028,...,-0.000190,-0.000101,-0.000052,-32.190122,-42.986946,-0.023471,0.427829,-0.003950,-1.147692,10.796824
2022-11-14 20:18:00,0.751147,0.000294,-0.718221,0.000422,-0.000253,-0.000242,-0.000069,0.00002,0.000459,-0.000003,...,-0.000180,-0.000098,-0.000051,-27.518817,-39.893321,-0.009338,0.437425,0.011163,14.748759,12.374503
2022-11-14 20:19:00,-0.154670,0.000703,1.388568,0.000675,-0.000139,-0.000222,-0.000065,0.00002,0.000520,0.000051,...,-0.000160,-0.000093,-0.000050,-22.633165,-36.441289,0.002610,0.524746,0.003284,-1.205912,13.808125


In [17]:
import pandas as pd
pd.set_option('display.max_columns', None)

df_final.describe()

,volume,mid_price,log_returns,7min_MA,25min_MA,99min_MA,300min_MA,900min_MA,7_ema,25_ema,99_ema,300_ema,900_ema,log_7min_MA,log_25min_MA,log_99min_MA,log_300min_MA,log_900min_MA,log_7_ema,log_25_ema,log_99_ema,log_300_ema,log_900_ema,MACD,Signal_Line,obv,rsi,vma,vol_pct,MACD-SIGNAL
count,524354.000000,524354.000000,5.241160e+05,5.243540e+05,5.243540e+05,5.243540e+05,5.243540e+05,5.243540e+05,524354.000000,5.243540e+05,5.243540e+05,5.243540e+05,5.243540e+05,5.243550e+05,5.243550e+05,5.243550e+05,5.243550e+05,5.243550e+05,524355.000000,5.243550e+05,5.243550e+05,5.243550e+05,5.243550e+05,524355.000000,524355.000000,524355.000000,524355.000000,524354.000000,524353.000000,524355.000000
mean,0.738366,0.000002,4.692463e+06,1.572959e-06,1.550584e-06,1.520459e-06,1.510905e-06,1.512507e-06,0.000002,1.549527e-06,1.526934e-06,1.512648e-06,1.521293e-06,1.550183e-06,1.543221e-06,1.518278e-06,1.510235e-06,1.512586e-06,0.000002,1.542826e-06,1.524792e-06,1.511861e-06,1.502882e-06,0.269798,0.269332,-0.607558,0.503172,0.003597,-2.957773,0.000467
std,7.659794,0.000478,8.329845e+09,2.139685e-04,1.141157e-04,5.946059e-05,3.291236e-05,1.913174e-05,0.000211,1.135725e-04,5.964812e-05,3.277820e-05,1.932648e-05,2.139669e-04,1.141065e-04,5.945459e-05,3.290945e-05,1.911925e-05,0.000211,1.135602e-04,5.962888e-05,3.274432e-05,1.926707e-05,17.093561,16.103947,2.570524,0.190887,0.097834,867.868641,5.128008
min,-0.998152,-0.033831,-2.097155e+12,-1.087993e-02,-3.116766e-03,-9.335592e-04,-2.972316e-04,-1.236377e-04,-0.013976,-5.183336e-03,-1.663310e-03,-5.505826e-04,-2.076951e-04,-1.109981e-02,-3.224901e-03,-9.679269e-04,-3.094243e-04,-1.290082e-04,-0.014350,-5.349346e-03,-1.727730e-03,-5.743286e-04,-2.178140e-04,-460.909365,-341.306509,-4.960160,0.000000,-0.844168,-546203.990281,-225.095058
25%,-0.452072,-0.000161,-1.689928e+00,-7.290454e-05,-3.764638e-05,-1.893669e-05,-1.051569e-05,-6.761935e-06,-0.000069,-3.595319e-05,-1.770287e-05,-9.621674e-06,-5.998760e-06,-7.291082e-05,-3.764836e-05,-1.893525e-05,-1.051646e-05,-6.759923e-06,-0.000069,-3.593404e-05,-1.767638e-05,-9.597165e-06,-5.972768e-06,-5.165284,-4.838255,-2.605714,0.363514,-0.027491,-2.644680,-1.765491
50%,-0.018714,0.000000,-7.457880e-01,8.827962e-07,5.829314e-07,4.064922e-07,2.346703e-07,2.527129e-07,0.000001,7.305376e-07,4.216444e-07,2.725259e-07,1.303000e-07,8.827089e-07,5.827691e-07,4.065534e-07,2.347278e-07,2.528640e-07,0.000001,7.495330e-07,4.417662e-07,2.927501e-07,1.474810e-07,0.087340,0.066665,-0.524102,0.501135,-0.000554,-1.261113,-0.004226
75%,0.782056,0.000164,2.270518e-01,7.571870e-05,3.959436e-05,2.064696e-05,1.136316e-05,7.643005e-06,0.000072,3.817698e-05,1.935707e-05,1.042115e-05,6.868256e-06,7.571733e-05,3.959676e-05,2.065042e-05,1.136211e-05,7.647534e-06,0.000072,3.819838e-05,1.938311e-05,1.044836e-05,6.881684e-06,5.509473,5.204620,1.661244,0.642901,0.025891,-0.518518,1.768885
max,4796.874745,0.033368,2.342612e+12,9.763860e-03,3.085761e-03,1.049758e-03,3.681025e-04,1.553923e-04,0.013695,5.004469e-03,1.725989e-03,6.069209e-04,2.380470e-04,9.586431e-03,2.998128e-03,1.039921e-03,3.559441e-04,1.501581e-04,0.013352,4.852726e-03,1.663965e-03,5.814832e-04,2.264805e-04,534.245074,399.719208,4.608425,1.000000,9.349762,149178.629496,236.834237


In [18]:
df_final.to_csv('C:\\Users\\user\\Documents\\GitHub\\crypto-scalping-RL-Agent\\data\\final_df.csv')

In [19]:
with open('C:\\Users\\user\\Documents\\GitHub\\crypto-scalping-RL-Agent\\data\\df_final.pkl', 'wb') as f:
    pickle.dump(df_final, f)


In [20]:
import numpy as np
from tqdm import tqdm


# 주어진 데이터프레임
sequence_length = 64
n_samples = len(df_final) - sequence_length + 1

# Extract the data as a NumPy array
data_array = df_final.to_numpy()

# Create an empty array to store the input sequences
input_sequences = np.empty((n_samples, (df_final.shape[1]), sequence_length))

for i in tqdm(range(n_samples)):
    input_seq = data_array[i:i + sequence_length, :].T  # Transpose to have (24, 60) shape
    input_sequences[i] = input_seq

print(input_sequences.shape)

100%|██████████| 524292/524292 [00:02<00:00, 220845.89it/s]

(524292, 30, 64)


In [21]:
with open('C:\\Users\\user\\Documents\\GitHub\\crypto-scalping-RL-Agent\\data\\binance_data.pkl', 'wb') as f:
    pickle.dump(input_sequences, f)
